<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 4 / SPRINT 3 / NOTE 2*

---

# N432. Beyond Classification

## 문제1.
Lecture Note에 구성되어 있는 U-Net모델에서 baseline 모델을 MobilenetV2가 아닌 Vgg16을 사용하겠습니다.

또한, 업샘플러 부분을 pix2pix가 아닌 직접,Conv2DTranspose를 사용해 구현하고 Sequential API가 아닌 함수형 API를 이용해보겠습니다.

U-Net을 구성하기 위해서는 vgg16 모델에서 다운샘플링할 때 꺼내온 5개의 레이어와 업샘플링할 때 레이어의 결과값의 형태가 같아야 합니다.

U-Net을 구성하기 위해 알맞은 A, B, C, D를 입력하세요.
- [100, 100, 100, 100] 형태로 입력하세요

In [2]:
import tensorflow as tf

In [10]:
img_shape = (128, 128, 3)
base_model = tf.keras.applications.VGG16(input_shape=img_shape, include_top=False)
down_stack=tf.keras.Model(inputs=[base_model.input],
                       outputs=[
                                base_model.get_layer(name='block5_conv3').output,
                                base_model.get_layer(name='block4_conv3').output,
                                base_model.get_layer(name='block3_conv3').output,
                                base_model.get_layer(name='block2_conv2').output,
                                base_model.get_layer(name='block1_conv2').output
])

down_stack.trainable = False

i=tf.keras.Input(shape=img_shape)

out, out1, out2, out3, out4 = down_stack(i)

A = 512
B = 256
C = 128
D = 64

out = tf.keras.layers.Conv2DTranspose(A, 3,strides=2,padding='same')(out)
out = tf.keras.layers.Add()([out,out1])

out = tf.keras.layers.Conv2DTranspose(B, 3,strides=2,padding='same')(out)
out = tf.keras.layers.Add()([out,out2])

out = tf.keras.layers.Conv2DTranspose(C, 3,strides=2,padding='same')(out)
out = tf.keras.layers.Add()([out,out3])

out = tf.keras.layers.Conv2DTranspose(D, 3,strides=2,padding='same')(out)
out = tf.keras.layers.Add()([out,out4])

out = tf.keras.layers.Conv2D(3, 3, activation='elu', padding='same') (out)
out = tf.keras.layers.Dense(3,activation='softmax')(out)

unet_model = tf.keras.Model(inputs=[i], outputs=[out])

In [4]:
out, out1, out2, out3, out4

(<KerasTensor: shape=(None, 8, 8, 512) dtype=float32 (created by layer 'model')>,
 <KerasTensor: shape=(None, 16, 16, 512) dtype=float32 (created by layer 'model')>,
 <KerasTensor: shape=(None, 32, 32, 256) dtype=float32 (created by layer 'model')>,
 <KerasTensor: shape=(None, 64, 64, 128) dtype=float32 (created by layer 'model')>,
 <KerasTensor: shape=(None, 128, 128, 64) dtype=float32 (created by layer 'model')>)

In [11]:
out, out1, out2, out3, out4

(<KerasTensor: shape=(None, 128, 128, 3) dtype=float32 (created by layer 'dense')>,
 <KerasTensor: shape=(None, 16, 16, 512) dtype=float32 (created by layer 'model_6')>,
 <KerasTensor: shape=(None, 32, 32, 256) dtype=float32 (created by layer 'model_6')>,
 <KerasTensor: shape=(None, 64, 64, 128) dtype=float32 (created by layer 'model_6')>,
 <KerasTensor: shape=(None, 128, 128, 64) dtype=float32 (created by layer 'model_6')>)